# Premier League Standings Data Retrieval and Database Integration

This code fetches Premier League standings data from the API, creates an SQLite database, and inserts the retrieved data into the "teams" and "stats" tables.

Importing necessary modules:

In [2]:
import requests
import sqlite3

Setting up the API URL and headers:

In [3]:
url = "https://premier-league-standings1.p.rapidapi.com/"

headers = {
    "X-RapidAPI-Key": "d5c307c9d2msh171b0eb139794dcp123df6jsnb413743e6dfe",
    "X-RapidAPI-Host": "premier-league-standings1.p.rapidapi.com"
}

Making a GET request to the API:

In [4]:
response = requests.get(url, headers=headers)
print(response.json())

[{'team': {'name': 'Manchester City', 'logo': 'https://a.espncdn.com/i/teamlogos/soccer/500/382.png', 'abbreviation': 'MNC'}, 'stats': {'wins': 28, 'losses': 5, 'ties': 5, 'gamesPlayed': 38, 'goalsFor': 94, 'goalsAgainst': 33, 'points': 89, 'rank': 1, 'goalDifference': 61}}, {'team': {'name': 'Arsenal', 'logo': 'https://a.espncdn.com/i/teamlogos/soccer/500/359.png', 'abbreviation': 'ARS'}, 'stats': {'wins': 26, 'losses': 6, 'ties': 6, 'gamesPlayed': 38, 'goalsFor': 88, 'goalsAgainst': 43, 'points': 84, 'rank': 2, 'goalDifference': 45}}, {'team': {'name': 'Manchester United', 'logo': 'https://a.espncdn.com/i/teamlogos/soccer/500/360.png', 'abbreviation': 'MAN'}, 'stats': {'wins': 23, 'losses': 9, 'ties': 6, 'gamesPlayed': 38, 'goalsFor': 58, 'goalsAgainst': 43, 'points': 75, 'rank': 3, 'goalDifference': 15}}, {'team': {'name': 'Newcastle United', 'logo': 'https://a.espncdn.com/i/teamlogos/soccer/500/361.png', 'abbreviation': 'NEW'}, 'stats': {'wins': 19, 'losses': 5, 'ties': 14, 'gamesP

Checking if the response was successful (status code 200):

In [5]:
if response.status_code == 200:
    premier_league_data = response.json()
else:
    print('Error retrieving fixture data. Status code:', response.status_code)

Creating a connection to the SQLite database:

In [6]:
conn = sqlite3.connect('results.db')
cursor = conn.cursor()

Dropping existing "teams" and "stats" tables (if they exist):

In [7]:
cursor.execute("DROP TABLE IF EXISTS teams")
cursor.execute("DROP TABLE IF EXISTS stats")

Creating the "teams" table if it doesn't exist:

In [8]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS teams (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        logo TEXT NOT NULL,
        abbreviation TEXT NOT NULL
    )
''')

Extracting team data from the API response and inserting it into the "teams" table:

In [9]:
team_data = []
for item in premier_league_data:
    team = item['team']
    name = team['name']
    logo = team['logo']
    abbreviation = team['abbreviation']
    team_data.append((name, logo, abbreviation))

cursor.executemany('INSERT INTO teams (name, logo, abbreviation) VALUES (?, ?, ?)', team_data)

Creating the "stats" table if it doesn't exist:

In [10]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stats (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        wins INTEGER NOT NULL,
        losses INTEGER NOT NULL,
        ties INTEGER NOT NULL,
        gamesPlayed INTEGER NOT NULL,
        goalsFor INTEGER NOT NULL,
        goalsAgainst INTEGER NOT NULL,
        points INTEGER NOT NULL,
        rank INTEGER NOT NULL,
        goalDifference INTEGER NOT NULL
    )
''')

Extracting team statistics from the API response and inserting them into the "stats" table:

In [11]:
team_stats = []
for item in premier_league_data:
    stats = item['stats']
    wins = stats['wins']
    losses = stats['losses']
    ties = stats['ties']
    gamesPlayed = stats['gamesPlayed']
    goalsFor = stats['goalsFor']
    goalsAgainst = stats['goalsAgainst']
    points = stats['points']
    rank = stats['rank']
    goalDifference = stats['goalDifference']
    team_stats.append((wins, losses, ties, gamesPlayed, goalsFor, goalsAgainst, points, rank, goalDifference))

cursor.executemany('INSERT INTO stats (wins, losses, ties, gamesPlayed, goalsFor, goalsAgainst, points, rank, '
                   'goalDifference) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', team_stats)


Committing the changes to the database and closing the connection:

In [12]:
conn.commit()
conn.close()